# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem. We can state our problem as labelling students as 'needing intervention' or 'not needing intervention'. This set of discrete labels that need to be predicted makes this a classification problem. We could also try to predict the probability of success for the students (which could be regression), but main interest here is in identifying the students needing help, so labelling them is sufficient here. There is always a chance that there could be further distinction within students who need help, (For example there could be students who need urgent intervention and students who cannot be helped at all), but since that is not stated in the problem, model does not have to find such sub categories. Also from the dataset, the target variable is clearly is a categorical variable making this, much more clearly, a classification problem. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [4]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


In [5]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = n_passed*100/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [6]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]


# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [7]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [8]:
X_all.head()

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1,0,1,0,18,0,1,1,0,1,...,1,0,0,4,3,4,1,1,3,6
1,1,0,1,0,17,0,1,1,0,0,...,1,1,0,5,3,3,1,1,3,4
2,1,0,1,0,15,0,1,0,1,0,...,1,1,0,4,3,2,2,3,3,10
3,1,0,1,0,15,0,1,1,0,0,...,1,1,1,3,2,2,1,1,5,2
4,1,0,1,0,16,0,1,1,0,0,...,1,0,0,4,3,2,1,2,5,4


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [15]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all,train_size=0.76,random_state=24)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

print "Grad rates:\nAll Data set has {:.2f}% yes instances".format(100 * (y_train == 'yes').mean())
print "Train set has {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Test set has {:.2f}%".format(100 * (y_test == 'yes').mean())

X_train.head()

Training set has 300 samples.
Testing set has 95 samples.
Grad rates:
All Data set has 67.00% yes instances
Train set has 67.00%
Test set has 67.37%


,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
71,1,0,0,1,15,0,1,1,0,0,...,1,1,0,3,3,3,1,1,3,0
119,1,0,0,1,15,0,1,1,0,0,...,1,1,0,3,4,3,1,2,4,6
151,1,0,0,1,16,0,1,0,1,0,...,1,0,1,4,4,4,3,5,5,6
17,1,0,1,0,16,0,1,1,0,0,...,1,0,0,5,3,2,1,1,4,4
364,0,1,1,0,17,1,0,1,0,0,...,1,0,0,3,2,2,1,2,3,0


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

I have chosen the following three models.
1. SVM is one of the highly accurate classifiers and works very well when the data is linearly 
   separable. In cases it is not, it is still applicable due to the kernel trick. The main 
   issue with this method is the training complexity, but given the small size of data here 
   that is not really an issue. The linear separability of data in high dimensions is not something that is easy to visually verify, so I am just taking it on faith that the data is. 

2. Random Forest is an emsemble method which combines predictions from  several simple decision trees to make the prediction. When the components decision trees are independent, it has a very good accuracy. Given that this problem has many categorical features (after conversion), the decision trees have several features to split on, allowing them to be independent. So the Random Forest should work well with this data. 
3. Naive bayes Method: This method makes a naive assumption that the features are independent of each other.
     This assumption is clearly not applicable here. For example, family size has some effect on the
     amount of time one has to study. Though the assumptions dont hold, thus the naive name, the method 
    performs quite well in practice. I also picked this model because I wanted to see an example of Bayesian learning and how it works with this given data. 
    

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import ShuffleSplit


# TODO: Import 'r2_score'
import sklearn.metrics as metrics

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = metrics.accuracy_score(y_true,y_predict)
    
    # Return the score
    return score


def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(X.shape[0], test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor = DecisionTreeClassifier()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = [{'max_depth' : [1,2,3,4,5,6,7,8,9,10]}]

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search object
    grid = GridSearchCV(regressor,param_grid=params,scoring=scoring_fnc,cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [11]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Applying decision tree 
clf = DecisionTreeClassifier()
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "Accuracy score for training examples %d is %f" %(size, acc)
# Produce the value for 'max_depth'

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "NB Accuracy score for training examples %d is %f" %(size, acc)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "RF Accuracy score for training examples %d is %f" %(size, acc)


from sklearn import svm
clf = svm.SVC()  
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "SVM Accuracy score for training examples %d is %f" %(size, acc)


confm = confusion_matrix(y_test, pred)
print confm
plt.matshow(confm)
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')

from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "KNN Accuracy score for training examples %d is %f" %(size, acc)


from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
for size in [100,200,300]:
    clf.fit(X_train[:size],y_train[:size])
    pred = clf.predict(X_test)
    acc = metrics.accuracy_score(pred, y_test)
    print "GB Accuracy score for training examples %d is %f" %(size, acc)

plt.show()


Accuracy score for training examples 100 is 0.600000
Accuracy score for training examples 200 is 0.652632
Accuracy score for training examples 300 is 0.694737
NB Accuracy score for training examples 100 is 0.368421
NB Accuracy score for training examples 200 is 0.642105
NB Accuracy score for training examples 300 is 0.705263
RF Accuracy score for training examples 100 is 0.694737
RF Accuracy score for training examples 200 is 0.757895
RF Accuracy score for training examples 300 is 0.715789
SVM Accuracy score for training examples 100 is 0.694737
SVM Accuracy score for training examples 200 is 0.715789
SVM Accuracy score for training examples 300 is 0.705263
[[ 7 22]
 [ 6 60]]
KNN Accuracy score for training examples 100 is 0.631579
KNN Accuracy score for training examples 200 is 0.610526
KNN Accuracy score for training examples 300 is 0.610526
GB Accuracy score for training examples 100 is 0.642105
GB Accuracy score for training examples 200 is 0.757895
GB Accuracy score for training e

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** I have chosen the following three models.
1. SVM is one of the highly accurate classifiers and works very well when the data is linearly 
   separable. In cases it is not, it is still applicable due to the kernel trick. The main 
   issue with this method is the training complexity, but given the small size of data here 
   that is not really an issue. Also given the smaller trainng size, the SVM may not learn the model well, i.e. high variance in the model. The linear separability of data in high dimensions is not something that is easy to visually verify, so I am just taking it on faith that the data is.  SVM is used for many classifier applications including document classification, Facial Recognition etc (http://www.clopinet.com/isabelle/Projects/SVM/applist.html).

2. Random Forest is an emsemble method which combines predictions from  several simple decision trees to make the prediction. When the component decision trees are independent, it has a very good accuracy. Given that this problem has many categorical features (after conversion), the decision trees have several features to split on, allowing them to be independent. So the Random Forest should work well with this data. Random Forests are used for classification and Regression applications. One application in our industry is in building a customer retention model using Random Forests. (http://dl.acm.org/citation.cfm?id=1707571) Random Forest methods as implemented in R also tell us which variables are important in predicting the response variable. 
3. Naive bayes Method: This method makes a naive assumption that the features are independent of each other.
     This assumption is clearly not applicable here. For example, family size has some effect on the
     amount of time one has to study. Though the assumptions dont hold, thus the naive name, the method 
    performs quite well in practice. I also picked this model because I wanted to see an example of Bayesian learning and how it works with this given data. Naive Bayes is typical used in the text classification. One classic application for NB is spam classification.    

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [18]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [19]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

clf_A = GaussianNB()
clf_B = RandomForestClassifier(random_state=24)
clf_C = svm.SVC(random_state=24)  

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.2791.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.2250.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0008 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.7909.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6769.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0008 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.7775.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6822.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0276 seconds
Made predictions in 0.0074 seconds.
F1 score for training set: 0.9867.
Made predictions in 0.0059 seconds.
F1 score for test set: 0.7778.
Training a RandomForestClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0008              |        0.0003          |      0.2791      |  0.2250               |
| 200               |      0.0010         |            0.0003            |     0.7909             |   0.6769              |
| 300               |         0.0008                |     0.0002                   |    0.7775              |    0.6822      |

** Classifer 2 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.2413                    |     0.0527                   |    0.9867              |      0.7778           |
| 200               |     0.2462             |        0.0542               |      0.9927            |     0.7536            |
| 300               |      0.2593                   |     0.0549                  |   0.9926               |     0.7385     |

** Classifer 3 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0011                    |        0.0007                |      0.8916            |      0.7898           |
| 200               |       0.0037                  |     0.0025                   |    0.8918              |      0.8105          |
| 300               |    0.0059                     |    0.0045                    |    0.8816              |     0.7974            |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** SVM clearly performs better on the test set, but I would choose the Random Forest as the appropriate model for predicting this data. The training F1 is very high indicating that the model has learned the training data too well, indicating a overfit situation. This overfitting situation shows up in weaker test results observed. 
By decreasing model complexity by parameter tuning, this overfit situation could be corrected. I also notice that SVM test error seems to have saturated indicating that model is not getting better with more training. The RF model clearly is a bit more expensive in terms of training as it is building about 100 decision trees for the ensemble. The training cost can be hidden from the consumers as the model can be trained offline. The test time performance is also a bit slow compared to other algorithms, but given the high accuracy that results from the model, it is a good trade off.  

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*


**Answer: ** Random Forest is a ensemble method which makes predictions based on several simple classifiers to make a prediction about an example. Each simple classifier in this case makes simple decisions to decide whether a student has passed or not. For example a simple decision rule would be that student will graduate if his/her study time is greater than some value S and if her attendance is greater than some value A. There are 30 such factors based on which decision rules can be formed. (Such factors are called features). Random Forest builds several such simple classifiers (called decision trees) using random set of features. To predict whether a student would graduate, the Random Forest uses these simple classifiers to make a predict and then takes a majority vote to make its prediction. For example if Random Forest used 100 simple classifiers, and for a student 75 of the classifiers predict passed = 'yes' and 25 predict passed = 'no', then Random Forest will predict that student will pass. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [28]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {'n_estimators': [10,50,100,200,300,400,500], 'max_depth':[1,2,3,4,5,10,20]}

# TODO: Initialize the classifier
clf =  RandomForestClassifier(random_state=24)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='yes')


# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,param_grid=parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

from IPython.display import display
display(pd.DataFrame(grid_obj.grid_scores_))

# Get the estimator
clf = grid_obj.best_estimator_

print "The best parameters are ", grid_obj.best_params_
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


/Library/Python/2.7/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


,0,1,2
0,"{u'n_estimators': 10, u'max_depth': 1}",0.802514,"[0.814814814815, 0.79754601227, 0.795180722892]"
1,"{u'n_estimators': 50, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
2,"{u'n_estimators': 100, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
3,"{u'n_estimators': 200, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
4,"{u'n_estimators': 300, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
5,"{u'n_estimators': 400, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
6,"{u'n_estimators': 500, u'max_depth': 1}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
7,"{u'n_estimators': 10, u'max_depth': 2}",0.798185,"[0.794871794872, 0.792452830189, 0.807228915663]"
8,"{u'n_estimators': 50, u'max_depth': 2}",0.799192,"[0.795180722892, 0.8, 0.802395209581]"
9,"{u'n_estimators': 100, u'max_depth': 2}",0.804839,"[0.812121212121, 0.8, 0.802395209581]"


The best parameters are  {'n_estimators': 300, 'max_depth': 4}
Made predictions in 0.1523 seconds.
Tuned model has a training F1 score of 0.8608.
Made predictions in 0.1486 seconds.
Tuned model has a testing F1 score of 0.8129.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The training_F1 is 0.8608 and the testing_F1 score is 0.8129. The testing F1 score of the untuned classifier was 0.7385.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.